In [ ]:
#Importa a função que carrega dados de vinhos
from sklearn.datasets import load_wine

#Mostrando info sobre o dataset
load_wine()

In [ ]:
#Variáveis independentes
X = load_wine()['data']
#Classes de vinhos - target
y = load_wine()['target']
display(X)
colunas = load_wine()['feature_names']

In [ ]:
#Importar a bibloteca para tratamento dos dados (pandas)
import pandas as pd

#Criando o dataframe
df = pd.DataFrame(X, columns=colunas)
df['classes'] = y
df.head()

In [ ]:
df.shape

In [ ]:
#Balanceamento das classes
df['classes'].value_counts()

In [ ]:
df['classes'].value_counts(1)

In [ ]:
#Técnica de holdout - Divisão treino e teste
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
#Analisando o Stratify
pd.Series(y_train).value_counts(1)

In [ ]:
pd.Series(y_test).value_counts(1)

In [ ]:
#Dimensionamento das escalas
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

#Calcula os parametros da escala e aplicar ao dados de treino
X_train = scaler.fit_transform(X_train)
#Teste também deve ser escalonado
X_test = scaler.fit_transform(X_test)

In [ ]:
#Importando função de regressão logística
from sklearn.linear_model import LogisticRegression
#instanciando o modelo
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
#Testando o modelo
#Obtendo as probabilidades da classe prevista
y_pred_proba = model.predict(X_test)

#Obtendo as previsões do modelo
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import precision_score
#Qual a precisão do modelo?
precisao = precision_score(y_test, y_pred, average='macro')
print(precisao)

#O parâmetro average=macro é eficiente quando as classes estiverem balanceada,
# pois considera que cada classe contribui igualmente para a métrica final

In [ ]:
precisao = precision_score(y_test, y_pred, average='weighted')
print(precisao)

# Weighted fornece uma métrica agregada que leva em conta o desequilíbrio das classes

In [ ]:
from sklearn.metrics import accuracy_score
acuracia = accuracy_score(y_test, y_pred)
print(acuracia)

In [ ]:
#Cross validation
from sklearn.model_selection import StratifiedKFold
# Definindo o número de folds
k = 5
#Inicializando a função
fold = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
#Listas para armazenar as precisões e acurárias
precisoes = list()
acuracias = list()
scaler = StandardScaler()

#Transformar X e y em um Dataframe e um Series do pandas
X = df.drop(columns='classes', axis=1)
y = df['classes']

In [ ]:
#Início das iterações de treino e testes
#Será aplicado o método de split no objeto fold, que retornará uma lista com os índices das instâncias que pertencem ao conjunto de treino e outra com os índices das instâncias que pertencem ao conjunto de teste.

for k, (train_index, test_index) in enumerate(fold.split(X,y)):
    #Dividindo os dados em treino e teste
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    #Escalando os dados. Todas as colunas serão passadas para uma distribuição normal, garantido que as features estejam em uma mesma escala numérica
    
    #Instanciando e treinando o modelo
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    #Obtendo a probabilidade das classes previstas
    y_pred_proba = model.predict_proba(X_test)
    #Obtendo a previsão do modelo
    y_pred = model.predict(X_test)
    
    #Calculando a precisão e acurácia do modelo treinado
    precisao = precision_score(y_test, y_pred, average='weighted')
    acuracia = accuracy_score(y_test, y_pred)
    
    #Armazena os dados nas listas
    precisoes.append(precisao)
    acuracias.append(acuracia)
    
    #Exibindo as métricas de cada fold
    print("=-"*6 + f"Fold: {k+1}" + "-="*6)
    print(f"Precisão: {precisao:.3f}, Acurácia: {acuracia:.3f}")

In [22]:
#Calculando as médias
precisao = sum(precisoes) / len(precisoes)
acuracia = sum(acuracias) / len(acuracias)
print(f"Média da Precisão: {precisao:.3f}, Média da Acurácia: {acuracia:.3f}")

Média da Precisão: 0.951, Média da Acurácia: 0.950
